<a href="https://www.kaggle.com/code/parisrohan/rp-customer-segmentation?scriptVersionId=96390055" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/online-retail-ii-uci/online_retail_II.csv


## Load the data and perform initial analysis

In [2]:
#import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [3]:
#read the dataset and store it into a dataframe
df_data = pd.read_csv('/kaggle/input/online-retail-ii-uci/online_retail_II.csv')
df_data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
#check the number of rows and columns present
df_data.shape

(1067371, 8)

Our dataset has 1067371 rows and 8 columns

In [5]:
#Check the feature datatypes
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


**Observation**: Features having null values are - Description, Customer ID

In [6]:
#making sure the above observation is correct
df_data.isnull().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

### Feature Description
* InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.

* StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.

* Description: Product (item) name. Nominal.

* Quantity: The quantities of each product (item) per transaction. Numeric.

* InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.

* UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£).

* CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.

* Country: Country name. Nominal. The name of the country where a customer resides.

### Analyzing Country feature

In [7]:
#Countries where the customers reside
df_data['Country'].value_counts()

United Kingdom          981330
EIRE                     17866
Germany                  17624
France                   14330
Netherlands               5140
Spain                     3811
Switzerland               3189
Belgium                   3123
Portugal                  2620
Australia                 1913
Channel Islands           1664
Italy                     1534
Norway                    1455
Sweden                    1364
Cyprus                    1176
Finland                   1049
Austria                    938
Denmark                    817
Unspecified                756
Greece                     663
Japan                      582
Poland                     535
USA                        535
United Arab Emirates       500
Israel                     371
Hong Kong                  364
Singapore                  346
Malta                      299
Iceland                    253
Canada                     228
Lithuania                  189
RSA                        169
Bahrain 

**Observation** : There are 756 entries with Country name as Unspecified.
This means that these are actually EMPTY/NULL values.

In [8]:
#We will delete records having 'Unspecified' country
index_name = df_data[df_data['Country']=='Unspecified'].index
df_data.drop(index_name, inplace=True)

In [9]:
print('Top 10 countries where the customers belong from are:', df_data['Country'].value_counts().index[:10])

Top 10 countries where the customers belong from are: Index(['United Kingdom', 'EIRE', 'Germany', 'France', 'Netherlands', 'Spain',
       'Switzerland', 'Belgium', 'Portugal', 'Australia'],
      dtype='object')


In [10]:
print('Countries from which least number of customers visit are:', df_data['Country'].value_counts().index[-10:])

Countries from which least number of customers visit are: Index(['Brazil', 'Thailand', 'Korea', 'European Community', 'Lebanon',
       'West Indies', 'Bermuda', 'Nigeria', 'Czech Republic', 'Saudi Arabia'],
      dtype='object')


### Analyzing Customer ID feature

In [11]:
#From the df_info() command we can see that the 'Customer ID' feature has some null values
print('Number of customers having no ID: ',df_data['Customer ID'].isnull().sum())

Number of customers having no ID:  242775


In [12]:
#Top 10 customers
df_data['Customer ID'].value_counts().head(10)

17841.0    13097
14911.0    11613
12748.0     7307
14606.0     6709
14096.0     5128
15311.0     4717
14156.0     4130
14646.0     3890
13089.0     3438
16549.0     3255
Name: Customer ID, dtype: int64

In [13]:
#We will handle the missing values in Customer ID feature by replacing them with value 0
df_data['Customer ID'] = df_data['Customer ID'].fillna(0)

### Analyzing Price feature

In [14]:
#As per our understanding, the Price feature does not contain any NULL values
#Check if there is any record with Price = 0
df_data[df_data['Price']==0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.0,0.0,United Kingdom
283,489463,71477,short,-240,2009-12-01 10:52:00,0.0,0.0,United Kingdom
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.0,0.0,United Kingdom
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.0,0.0,United Kingdom
3114,489655,20683,NaN,-44,2009-12-01 17:26:00,0.0,0.0,United Kingdom
...,...,...,...,...,...,...,...,...
1062442,581234,72817,NaN,27,2011-12-08 10:33:00,0.0,0.0,United Kingdom
1063965,581406,46000M,POLYESTER FILLER PAD 45x45cm,240,2011-12-08 13:58:00,0.0,0.0,United Kingdom
1063966,581406,46000S,POLYESTER FILLER PAD 40x40cm,300,2011-12-08 13:58:00,0.0,0.0,United Kingdom
1064015,581408,85175,NaN,20,2011-12-08 14:06:00,0.0,0.0,United Kingdom


**Observation**: The above 6202 record's Price feature has a 0 value. This can be treated as NULL 

In [15]:
#Check if there are any negative values in Price
df_data[df_data['Price']<0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
179403,A506401,B,Adjust bad debt,1,2010-04-29 13:36:00,-53594.36,0.0,United Kingdom
276274,A516228,B,Adjust bad debt,1,2010-07-19 11:24:00,-44031.79,0.0,United Kingdom
403472,A528059,B,Adjust bad debt,1,2010-10-20 12:04:00,-38925.87,0.0,United Kingdom
825444,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,0.0,United Kingdom
825445,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,0.0,United Kingdom


### Analyzing the InvoiceDate feature

In [16]:
#data start
df_data['InvoiceDate'].min()

'2009-12-01 07:45:00'

In [17]:
#data stop
df_data['InvoiceDate'].max()

'2011-12-09 12:50:00'

### Analyzing the Quantity feature

In [18]:
#Check if there are any values as 0
df_data[df_data['Quantity']==0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country


In [19]:
df_data[df_data['Quantity']<0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
...,...,...,...,...,...,...,...,...
1065910,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
1067002,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
1067176,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
1067177,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


**Observation**: There are 22950 records with negative quantity. This is because the corresponding Invoice entry has been canceled. 


### Analyzing the Description feature

In [20]:
df_data[df_data['Description'].isna()]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.0,0.0,United Kingdom
3114,489655,20683,NaN,-44,2009-12-01 17:26:00,0.0,0.0,United Kingdom
3161,489659,21350,NaN,230,2009-12-01 17:39:00,0.0,0.0,United Kingdom
3731,489781,84292,NaN,17,2009-12-02 11:45:00,0.0,0.0,United Kingdom
4296,489806,18010,NaN,-770,2009-12-02 12:42:00,0.0,0.0,United Kingdom
...,...,...,...,...,...,...,...,...
1060783,581199,84581,NaN,-2,2011-12-07 18:26:00,0.0,0.0,United Kingdom
1060787,581203,23406,NaN,15,2011-12-07 18:31:00,0.0,0.0,United Kingdom
1060793,581209,21620,NaN,6,2011-12-07 18:35:00,0.0,0.0,United Kingdom
1062442,581234,72817,NaN,27,2011-12-08 10:33:00,0.0,0.0,United Kingdom


In [21]:
#We will drop the rows where Description is NULL && Price = 0 & Customer ID = 0
index_names = df_data[(df_data['Description'].isna()) & (df_data['Price']==0) & (df_data['Customer ID']==0)].index

df_data.drop(index_names, inplace = True)

### Analyzing Invoice

In [22]:
#We will drop the records who have been cancelled -> Invoice start with C
index_name = df_data[(df_data['Quantity']<0)].index
df_data.drop(index_name, inplace = True)

In [23]:
index_names = df_data[(df_data['Price']==0) & (df_data['Customer ID']==0)].index
print(index_names)

Int64Index([   5904,   31993,   63439,   77965,   99948,   99981,  111414,
             111415,  111416,  111417,
            ...
            1047618, 1047706, 1047707, 1049836, 1049935, 1050083, 1060786,
            1060795, 1063965, 1063966],
           dtype='int64', length=981)


In [24]:
#data after handling null values
df_data2 = df_data.copy(deep=True)
df_data2.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
